Patient Journey Notebook
Neo4j GDS on Snowflake v0.3.11

Author: Dave Fauth

Last Updated: 1 May 2025

Neo4j Graph Analytics for Snowflake Version: 0.3.11

## Setting Up
Before we run our algorithms, we need to set the proper permissions. But before we get started granting different roles, we need to ensure that you are using `accountadmin` to grant and create roles. Lets do that now:

In [ ]:
-- you must be accountadmin to create role and grant permissions
USE ROLE accountadmin;

Create a database which we will use to prepare data for GDS.

In [ ]:
-- Create a database which we will use to prepare data for GDS.
CREATE DATABASE IF NOT EXISTS NEO4J_PATIENT_DB;
CREATE SCHEMA IF NOT EXISTS NEO4J_PATIENT_DB.PUBLIC;
USE SCHEMA NEO4J_PATIENT_DB.PUBLIC;

As part of the lab, we are going to create 2 tables for our source data:
 - `PATIENTS` -- The dataset that we will use for our patients data.
 - `PROCEDURES` -- The dataset containing the procedures performed on our patients.


In [ ]:
CREATE OR REPLACE TABLE NEO4J_PATIENT_DB.PUBLIC.PATIENTS (
	ID VARCHAR(16777216),
	PREFIX VARCHAR(16777216),
	FIRST VARCHAR(16777216),
	MIDDLE VARCHAR(16777216),
	LAST VARCHAR(16777216),
	SUFFIX VARCHAR(16777216),
	MAIDEN VARCHAR(16777216),
	MARITAL VARCHAR(16777216),
	RACE VARCHAR(16777216),
	ETHNICITY VARCHAR(16777216),
	GENDER VARCHAR(16777216),
	BIRTHPLACE VARCHAR(16777216),
	ADDRESS VARCHAR(16777216),
	CITY VARCHAR(16777216),
	STATE VARCHAR(16777216),
	ZIP NUMBER(38,0)
);

In [ ]:
CREATE OR REPLACE TABLE NEO4J_PATIENT_DB.PUBLIC.PROCEDURES (
	STARTDATE TIMESTAMP_NTZ(9),
	STOP TIMESTAMP_NTZ(9),
	PATIENT VARCHAR(16777216),
	ENCOUNTER VARCHAR(16777216),
	SYSTEM VARCHAR(16777216),
	CODE NUMBER(38,0),
	DESCRIPTION VARCHAR(16777216),
	BASE_COST NUMBER(38,2),
	REASONCODE NUMBER(38,0),
	REASONDESCRIPTION VARCHAR(16777216)
);

[Stages](https://docs.snowflake.com/en/user-guide/data-load-local-file-system-create-stage.html) in snowflake are places that you can land your data before it is uploaded to a Snowflake table. You might have a batch of CSV files living on a disk driver somewhere, and, in order to start querying the data via a table, the data must be landed within the Snowflake environment for a data upload to be possible.

In the exercise, we will be working with structured, comma-delimited data that has already been staged in a public, external AWS bucket. Before we can use this data, we first need to create a `Stage` that specifies the location of our external bucket.

In [ ]:
CREATE OR REPLACE STAGE sf_neo4j_data_stage
  URL = 's3://neo4j-snowflake-data/patient-journey/';

We also can take a look at the contents of the `sf_neo4j_data_stage` by executing the follow:

```bash
ls @sf_neo4j_data_stage;
```

In [ ]:
ls @sf_neo4j_data_stage;


[File Formats](https://docs.snowflake.com/en/sql-reference/sql/create-file-format.html) tell Snowflake the structure of the data coming in. The last thing that we need to do before we can load the data into our Snowflake tables is: we have to create a `File Format` that matches the data structure of the local files we want to upload. As smart as Snowflake is, its not THAT smart.

For our example, our data has header columns in the CSV, so we want to skip those. A comma delimiter is the default way to delimit CSV files (hence the name), but sometimes you can choose another character. We need to give Snowflake all the details on how we have organized our data in the files we want to load in. Please execute the following code:

In [ ]:
CREATE OR REPLACE FILE FORMAT patientdata_ff
  TYPE = 'CSV'
  COMPRESSION = 'AUTO'
  FIELD_DELIMITER = ','
  RECORD_DELIMITER = '\n'
  SKIP_HEADER = 1
  FIELD_OPTIONALLY_ENCLOSED_BY = 'NONE'
  TRIM_SPACE = FALSE
  ERROR_ON_COLUMN_COUNT_MISMATCH = TRUE
  ESCAPE = 'NONE'
  ESCAPE_UNENCLOSED_FIELD = '\134'
  DATE_FORMAT = 'AUTO'
  TIMESTAMP_FORMAT = 'AUTO'
  NULL_IF = ('\\N');

In this section, we will use a virtual [warehouse](https://docs.snowflake.com/en/user-guide/warehouses-overview.html) and the [COPY command](https://docs.snowflake.com/en/sql-reference/sql/copy-into-table.html) to initiate bulk loading of the CSV file sitting in our AWS external stage, moving it into the Snowflake table we just created.

We can run a COPY command to load the data into the `NEO4J_PATIENT_DB` database we created earlier. Go ahead and execute the next set of statements in the worksheet to load the staged data into the tables:


In [ ]:
COPY INTO PATIENTS FROM @sf_neo4j_data_stage/Patients.csv
  FILE_FORMAT = (FORMAT_NAME = patientdata_ff);

COPY INTO PROCEDURES FROM @sf_neo4j_data_stage/Procedures.csv
  FILE_FORMAT = (FORMAT_NAME = patientdata_ff);

We now finally have data inside a Snowflake table that is ready to be queried on demand. We can see a sample of what data lies within our table by execute the following:


In [ ]:
SELECT * FROM NEO4J_PATIENT_DB.PUBLIC.PROCEDURES LIMIT 10;

In [ ]:

SELECT * FROM NEO4J_PATIENT_DB.PUBLIC.PATIENTS LIMIT 10;

In [ ]:
CREATE OR REPLACE TABLE NEO4J_PATIENT_DB.PUBLIC.PATIENT_NODE_MAPPING (nodeId) AS
SELECT DISTINCT p.ID from NEO4J_PATIENT_DB.PUBLIC.PATIENTS p;

In [ ]:
CREATE OR REPLACE TABLE NEO4J_PATIENT_DB.PUBLIC.PROCEDURE_NODE_MAPPING (nodeId) AS
SELECT DISTINCT p.code from NEO4J_PATIENT_DB.PUBLIC.PROCEDURES p;

Since GDS uses an application, we are going to now setup the application roles and higher level privileges.



In [ ]:
USE ROLE ACCOUNTADMIN;
USE SCHEMA NEO4J_PATIENT_DB.PUBLIC;

-- Create a consumer role for users and admins of the GDS application
CREATE ROLE IF NOT EXISTS gds_role;
GRANT APPLICATION ROLE se_snow_neo4j_graph_analytics.app_user TO ROLE gds_role;
GRANT APPLICATION ROLE se_snow_neo4j_graph_analytics.app_admin TO ROLE gds_role;

-- Grant access to consumer data
GRANT USAGE ON DATABASE NEO4J_PATIENT_DB TO APPLICATION se_snow_neo4j_graph_analytics;
GRANT USAGE ON SCHEMA NEO4J_PATIENT_DB.PUBLIC TO APPLICATION se_snow_neo4j_graph_analytics;

-- Required to read tabular data into a graph
GRANT SELECT ON ALL TABLES IN SCHEMA NEO4J_PATIENT_DB.PUBLIC TO APPLICATION se_snow_neo4j_graph_analytics;

-- Required to write computation results into a table/view
GRANT CREATE TABLE ON SCHEMA NEO4J_PATIENT_DB.PUBLIC TO APPLICATION se_snow_neo4j_graph_analytics;
GRANT CREATE VIEW ON SCHEMA NEO4J_PATIENT_DB.PUBLIC TO APPLICATION se_snow_neo4j_graph_analytics;

-- Ensure the consumer role has access to created tables/views
GRANT ALL PRIVILEGES ON FUTURE TABLES IN SCHEMA NEO4J_PATIENT_DB.PUBLIC TO ROLE gds_role;
GRANT ALL PRIVILEGES ON ALL TABLES IN SCHEMA NEO4J_PATIENT_DB.PUBLIC TO ROLE gds_role;

GRANT CREATE TABLE ON SCHEMA NEO4J_PATIENT_DB.PUBLIC TO ROLE gds_role;
GRANT CREATE VIEW ON SCHEMA NEO4J_PATIENT_DB.PUBLIC TO ROLE gds_role;
GRANT CREATE VIEW ON SCHEMA NEO4J_PATIENT_DB.PUBLIC TO APPLICATION se_snow_neo4j_graph_analytics;

-- Compute and warehouse access
GRANT USAGE ON WAREHOUSE GDSONSNOWFLAKE TO APPLICATION se_snow_neo4j_graph_analytics;


In [ ]:

DROP TABLE IF EXISTS NEO4J_PATIENT_DB.PUBLIC.KP_Community;
DROP TABLE IF EXISTS NEO4J_PATIENT_DB.PUBLIC.PATIENT_PROCEDURE_SIMILARITY;


We will generate the patient similarity using a GDS Jaccard similarity algorithm and then we will generate communities using a GDS Louvain algorithm.

1. **patients** Unique Patient Nodes
2. **procedures** Unique Procedure Nodes
3. **patients - procedures** Patients that had Kidney related procedures

In [ ]:
// create a subset of patients that have had any of the 4 kidney disease codes
CREATE OR REPLACE VIEW KidneyPatients_vw (nodeId) AS
    SELECT DISTINCT PATIENT_NODE_MAPPING.NODEID as nodeId
    FROM PROCEDURES
            JOIN PATIENT_NODE_MAPPING ON PATIENT_NODE_MAPPING.NODEID = PROCEDURES.PATIENT 
    WHERE PROCEDURES.REASONCODE IN (431857002,46177005,161665007,698306007)
;

Next, create a view that represents a subset of procedures nodes in the graph.  
This is a subset of all procedures, but includes all procedures that kidney patients have had regardless of the reason.

In [ ]:
// There are ~400K procedures - it is doubtful that the kidney patients even have used a small
// fraction of those.  To reduce GDS memory and speed algorithm execution, we want to load
// only those procedures that kidney patients have had.
CREATE OR REPLACE VIEW KidneyPatientProcedures_vw (nodeId) AS
    SELECT DISTINCT PROCEDURE_NODE_MAPPING.NODEID as nodeId
    FROM PROCEDURES 
        JOIN PROCEDURE_NODE_MAPPING ON PROCEDURE_NODE_MAPPING.nodeId = PROCEDURES.CODE
        JOIN KIDNEYPATIENTS_VW ON PATIENT = PROCEDURES.PATIENT
;

Finally create a view that represents the relationship between the kidney patients and all the procedures they have had.  
This will be the relationship used in the bipartite graph projection for Jaccard similarity


In [ ]:
// create the relationship view of kidney patients to the procedures they have had
CREATE OR REPLACE VIEW KidneyPatientProcedure_relationship_vw (sourceNodeId, targetNodeId) AS
    SELECT DISTINCT PATIENT_NODE_MAPPING.NODEID as sourceNodeId, PROCEDURE_NODE_MAPPING.NODEID as targetNodeId
    FROM PATIENT_NODE_MAPPING
         JOIN PROCEDURES ON PROCEDURES.PATIENT = PATIENT_NODE_MAPPING.NODEID
         JOIN PROCEDURE_NODE_MAPPING ON PROCEDURE_NODE_MAPPING.NODEID = PROCEDURES.CODE
;

In [ ]:
USE ROLE ACCOUNTADMIN;
GRANT SELECT ON ALL VIEWS IN SCHEMA NEO4J_PATIENT_DB.PUBLIC TO APPLICATION se_snow_neo4j_graph_analytics;
GRANT SELECT ON ALL TABLES IN SCHEMA NEO4J_PATIENT_DB.PUBLIC TO APPLICATION se_snow_neo4j_graph_analytics;

We will switch to the gds_role.

In [ ]:
USE ROLE gds_role;

Next you will compute the Jaccard similarity and add those results to the in-memory graph (mutate).  The gds.node_similarity() function in GDS supports Jaccard (the default), Overlap or Cosine similarity methods. To reduce the noise in the graph as there will  likely be quite a bit of commonality for common conditions, you will use a similarity cutoff of 0.3 and for each node only consider the top 10 most similar patients.


In [ ]:
CALL se_snow_neo4j_graph_analytics.graph.node_similarity('CPU_X64_L', {
  'project': {
    'defaultTablePrefix': 'neo4j_patient_db.public',
    'nodeTables': ['KidneyPatients_vw','KidneyPatientProcedures_vw'], 
    'relationshipTables': {
      'KidneyPatientProcedure_relationship_vw': {
        'sourceTable': 'KidneyPatients_vw',
        'targetTable': 'KidneyPatientProcedures_vw'
      }
    }
  },
  'compute': { 'topK': 10,
                'similarityCutoff': 0.3,
                'similarityMetric': 'JACCARD'
            },
  'write': [
    {
    'sourceLabel':          'KidneyPatients_vw',
    'targetLabel':          'KidneyPatients_vw',
    'relationshipProperty': 'similarity',
    'outputTable':          'neo4j_patient_db.public.PATIENT_PROCEDURE_SIMILARITY'
    }
  ]
});

In [ ]:
SELECT SOURCENODEID, TARGETNODEID, SIMILARITY
FROM NEO4J_PATIENT_DB.PUBLIC.PATIENT_PROCEDURE_SIMILARITY 
LIMIT 10;

When you write the tables back to the database, the tables are owned by the application. You will need to alter the tables so that subsequent post-processing or visualization is easier by avoiding joins to the node ID mapping tables necessary for GDS. To do this, you have to transfer ownership of the tables back to accountadmin role.

In [ ]:
USE ROLE ACCOUNTADMIN;
GRANT OWNERSHIP ON TABLE NEO4J_PATIENT_DB.PUBLIC.PATIENT_PROCEDURE_SIMILARITY TO ROLE ACCOUNTADMIN REVOKE CURRENT GRANTS;
-- Required to read tabular data into a graph
GRANT SELECT ON ALL TABLES IN SCHEMA NEO4J_PATIENT_DB.PUBLIC. TO APPLICATION se_snow_neo4j_graph_analytics;
USE ROLE gds_role;

In [ ]:
CALL se_snow_neo4j_graph_analytics.graph.louvain('CPU_X64_L', {
  'project': {
    'defaultTablePrefix': 'neo4j_patient_db.public',
    'nodeTables': ['KidneyPatients_vw','KidneyPatientProcedures_vw'], 
    'relationshipTables': {
      'PATIENT_PROCEDURE_SIMILARITY': {
        'sourceTable': 'KidneyPatients_vw',
        'targetTable': 'KidneyPatients_vw'
      }
    }
  },
  'compute': { 'maxIterations': 50,
                'includeIntermediateCommunities': false,
                'relationshipWeightProperty': 'similarity'
            },
  'write': [
    {
    'nodeLabel':          'KidneyPatients_vw',
    'outputTable':        'NEO4J_PATIENT_PUBLIC.KP_Community'
    }
  ]
});

In [ ]:
USE ROLE ACCOUNTADMIN;
GRANT OWNERSHIP ON TABLE NEO4J_PATIENT_DB.PUBLIC.KP_Community TO ROLE ACCOUNTADMIN REVOKE CURRENT GRANTS;
USE ROLE gds_role;

Next you will alter the tables to add the patient ID.

In [ ]:
ALTER TABLE NEO4J_PATIENT_DB.PUBLIC.KP_Community ADD PATIENT VARCHAR(60);
ALTER TABLE NEO4J_PATIENT_DB.PUBLIC.PATIENT_PROCEDURE_SIMILARITY ADD SOURCE_PATIENT VARCHAR(60), TARGET_PATIENT VARCHAR(60);

UPDATE KP_Community
SET PATIENT = PATIENT_NODE_MAPPING.NODEID
FROM PATIENT_NODE_MAPPING
WHERE PATIENT_NODE_MAPPING.NODEID = KP_Community.NODEID
;
COMMIT
;

UPDATE PATIENT_PROCEDURE_SIMILARITY
SET SOURCE_PATIENT = PATIENT_NODE_MAPPING.NODEID
FROM PATIENT_NODE_MAPPING
WHERE PATIENT_NODE_MAPPING.NODEID = PATIENT_PROCEDURE_SIMILARITY.SOURCENODEID
;

UPDATE PATIENT_PROCEDURE_SIMILARITY
SET TARGET_PATIENT = PATIENT_NODE_MAPPING.NODEID
FROM PATIENT_NODE_MAPPING
WHERE PATIENT_NODE_MAPPING.NODEID = PATIENT_PROCEDURE_SIMILARITY.TARGETNODEID
;
COMMIT;

In [ ]:
SELECT NODEID, PATIENT, COMMUNITY
FROM NEO4J_PATIENT_DB.PUBLIC.KP_Community 
ORDER BY COMMUNITY ASC;
